# Facial Landmark Detection using OpenCV and Dlib

Include de neccesary dynamic libraries and headers

In [1]:
#include "include_libraries.h"

In [2]:
#include <string>
#include <cstring>
#include <fstream>

#include <dlib/opencv.h>
#include <dlib/image_processing.h>
#include <dlib/image_processing/frontal_face_detector.h>
#include <opencv2/opencv.hpp> 

using namespace std;
using namespace cv;
using namespace dlib;

In [3]:
// Write landmarks to file
void writeLandmarksToFile(dlib::full_object_detection &landmarks, const std::string &filename)
{
  // Open file
	std::ofstream ofs;
	ofs.open(filename);
  
  // Loop over all landmark points
  for (int i = 0; i < landmarks.num_parts(); i++)
	{
    // Print x and y coordinates to file
		ofs << landmarks.part(i).x() << " " << landmarks.part(i).y() << std::endl;

	}
  // Close file
	ofs.close();
}

Create a function to draw the landmarks in the image

In [4]:
void drawLandmarks(cv::Mat &im, dlib::full_object_detection landmarks )
{
  for(int i = 0; i < landmarks.num_parts(); i++)
  {
    int px = landmarks.part(i).x();
    int py = landmarks.part(i).y();

    char landmark_index[3];
    sprintf(landmark_index, "%d", i+1);

    circle(im, cv::Point(px, py), 1, cv::Scalar(0, 0, 255), 2, cv::LINE_AA);
    putText(im, landmark_index, cv::Point(px, py), cv::FONT_HERSHEY_SIMPLEX, .3, cv::Scalar(255, 0, 0), 1);
  }
}

Create the landmark detector and the face detector

In [5]:
dlib::frontal_face_detector faceDetector = dlib::get_frontal_face_detector();

In [6]:
dlib::shape_predictor landmarkDetector;

After that, deserialize the shape predictor for face landmarks into the landmark detector

In [7]:
dlib::deserialize("shape_predictor_68_face_landmarks.dat") >> landmarkDetector;

Read the image into a dlib image

In [8]:
std::string imageFilename("images/girl.jpg");
cv::Mat im = cv::imread(imageFilename);

std::string landmarksBasename("output");

dlib::cv_image<dlib::bgr_pixel> dlibIm(im);

Detect the faces in the image

In [9]:
std::vector<dlib::rectangle> faceRects = faceDetector(dlibIm);
std::cout << "Number of faces detected: " << faceRects.size() << std::endl;

Number of faces detected: 1


Detect the face landmarks in each of the faces detected

In [10]:
std::vector<dlib::full_object_detection> landmarksAll;
for (int i = 0; i < faceRects.size(); i++)
  {
    // For every face rectangle, run landmarkDetector
    dlib::full_object_detection landmarks;
    
    landmarks = landmarkDetector(dlibIm, faceRects[i]);
    
    // Print number of landmarks
    if (i == 0) std::cout << "Number of landmarks : " << landmarks.num_parts() << std::endl;
    
    // Store landmarks for current face
    landmarksAll.push_back(landmarks);
    
    // Draw landmarks on face
    drawLandmarks(im, landmarks);

    // Write landmarks to disk
    std::stringstream landmarksFilename;
    landmarksFilename << landmarksBasename <<  "_"  << i << ".txt";
    std::cout << "Saving landmarks to " << landmarksFilename.str() << std::endl;
    writeLandmarksToFile(landmarks, landmarksFilename.str());
    
  }

Number of landmarks : 68
Saving landmarks to output_0.txt


In [11]:
std::string outputFilename("images/girl_output.jpg");
std::cout << "Saving output image to " << outputFilename << std::endl;

Saving output image to images/girl_output.jpg


In [12]:
cv::imwrite(outputFilename, im);